In [6]:
from __future__ import division
import pickle
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd
import scipy
from scipy import stats

In [7]:
X_train = pd.read_csv("data/forty_column_dataset.csv")
X_test = pd.read_csv("data/X_priv.csv")
X_train_old = pd.read_csv("data/X_train.csv")

In [8]:
X_test.shape

(302047, 28)

In [9]:
del X_test['SelectedPackage']
# del X_test['TodayDate']
# del X_test['ReceivedDateTime']
del X_test['Unnamed: 0']

In [10]:
npp_unique = np.unique(X_train_old['NameOfPolicyProduct'])
updated_npp = np.array([np.arange(len(npp_unique))[np.array(val==npp_unique, dtype=bool)][0] for val in X_test['NameOfPolicyProduct']])
X_test['NameOfPolicyProduct'] = updated_npp

In [11]:
scid_unique = np.unique(X_train_old['SCID'])
updated_scid = np.array([len(scid_unique)+1 if val not in scid_unique else \
                         np.arange(len(scid_unique))[np.array(val==scid_unique, dtype=bool)][0] for val in X_test['SCID']])
X_test['SCID'] = updated_scid

/Users/saito/.local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [12]:
ids = np.array(X_test['CustomerMD5Key'])

In [13]:
a = scipy.stats.itemfreq(ids)
a_ids = np.array([x[0] for x in a])
a_freq = np.array([int(x[1]) for x in a])

keys = a_ids
values = a_freq
dictionary = dict(zip(keys, values))

In [14]:
customerkey_index = 0
pp_index = np.arange(len(X_test.columns))[np.array(X_test.columns) =='NameOfPolicyProduct' ][0]

master_pp = {i:[] for i in a_ids}

for idx, i in enumerate(np.array(X_test)):
    #if i[pp_index] not in master_pp[i[customerkey_index]]:
    master_pp[i[customerkey_index]].append(i[pp_index])
    
num_unique_pols = np.array([len(np.unique(master_pp.get(i))) for i in X_test['CustomerMD5Key']] )
num_total_pols =  np.array([len(master_pp.get(i)) for i in X_test['CustomerMD5Key']] )

In [15]:
cam_index = np.arange(len(X_test.columns))[np.array(X_test.columns) =='CarAnnualMileage'][0]
cmi_index = np.arange(len(X_test.columns))[np.array(X_test.columns) =='CarMakeId'][0]
ca_index = np.arange(len(X_test.columns))[np.array(X_test.columns) =='CarAge'][0]
civ_index = np.arange(len(X_test.columns))[np.array(X_test.columns) =='CarInsuredValue'][0]

master_cam = {i:[] for i in a_ids}
master_cmi = {i:[] for i in a_ids}
master_ca = {i:[] for i in a_ids}
master_civ = {i:[] for i in a_ids}

for idx, i in enumerate(np.array(X_test)):
    master_cam[i[customerkey_index]].append(i[cam_index])
    master_cmi[i[customerkey_index]].append(i[cmi_index])
    master_ca[i[customerkey_index]].append(i[ca_index])
    master_civ[i[customerkey_index]].append(i[civ_index])

In [16]:
num_unique_cam = np.array([len(np.unique(master_cam.get(i))) for i in X_test['CustomerMD5Key']] )
num_unique_cmi = np.array([len(np.unique(master_cmi.get(i))) for i in X_test['CustomerMD5Key']] )
num_unique_ca = np.array([len(np.unique(master_ca.get(i))) for i in X_test['CustomerMD5Key']] )
num_unique_civ = np.array([len(np.unique(master_civ.get(i))) for i in X_test['CustomerMD5Key']] )

In [17]:
car_con_1 = (num_unique_ca!=num_unique_civ)
car_con_2 = (num_unique_cam!=num_unique_civ)
car_con_3 = (num_unique_cmi!=num_unique_civ)
car_con_4 = (num_unique_cam!=num_unique_cmi)
car_con_5 = (num_unique_ca!=num_unique_cmi)
car_con_6 = (num_unique_cam!=num_unique_ca)

In [18]:
vendor_freq = scipy.stats.itemfreq(X_train['AffinityCodeId'])
log_vf = np.array([np.log10(x[1]) for x in vendor_freq])
vendor_keys = np.array([x[0] for x in vendor_freq])
vendor_values = [0 if x<1.5 else 1 if (x>=1.5)&(x<4) else 2 for x in log_vf]
vendor_dict  = dict(zip(vendor_keys, vendor_values))
vendor_log_sales = np.array([vendor_dict.get(x) for x in X_test['AffinityCodeId']])

In [19]:
vendor_values_unlogged = np.array([x[1] for x in vendor_freq])
success_mask = np.array(X_train['labels']==1)
vendor_success_scipy = scipy.stats.itemfreq(X_train['AffinityCodeId'][success_mask])
vendor_success_names = np.array([x[0] for x in vendor_success_scipy])
vendor_success_freqs = np.array([x[1] for x in vendor_success_scipy])
vendor_success_rates = np.array([0 if x not in vendor_success_names else \
                        vendor_success_freqs[vendor_success_names==x][0]/y  for x,y in zip(vendor_keys, vendor_values_unlogged)])
vendor_success_dict  = dict(zip(vendor_keys, vendor_success_rates))
vendor_success_final = np.array([vendor_success_dict.get(x) for x in X_test['AffinityCodeId']])

In [20]:
annual_milage_per_age = X_test['CarAnnualMileage']/(X_test['CarAge']+2)
voluntary_payment_per_value = X_test['VoluntaryExcess']/(X_test['CarInsuredValue']+1)

X_test['voluntary_payment_per_value'] = voluntary_payment_per_value
X_test['annual_milage_per_age'] = annual_milage_per_age
X_test['vendor_success_rate'] = vendor_success_final
X_test['vendor_log_sales'] = vendor_log_sales

X_test['age_minus_policy'] = X_test['FirstDriverAge'] - X_test['PolicyHolderNoClaimDiscountYears']

X_test['num_unique_pols'] = num_unique_pols
X_test['num_total_pols'] = num_total_pols 

X_test['car_flag_1'] = car_con_1
X_test['car_flag_2'] = car_con_2
X_test['car_flag_3'] = car_con_3
X_test['car_flag_4'] = car_con_4
X_test['car_flag_5'] = car_con_5
X_test['car_flag_6'] = car_con_6

In [21]:
X_test.replace([np.inf, -np.inf], np.nan)
X_test = X_test.fillna(method = 'ffill')

In [22]:
np.savez_compressed('juice_test.npz', X_test.ix[:, (X_test.columns != 'CustomerMD5Key')])

In [23]:
# X_train.columns

In [24]:
# X_test.ix[:, (X_test.columns != 'CustomerMD5Key')].columns

In [25]:
# X_test.shape

In [26]:
# X_train.shape

In [27]:
# plt.hist(X_train['SCID'])
# plt.show()
# plt.hist(X_test['SCID'])

In [28]:
forty_col_test = np.load('juice_test.npz')

In [29]:
# forty_col_test['arr_0'].shape()

In [30]:
X_test.to_csv("juice_test.csv")

In [31]:
c = X_test

In [32]:
# c.shape

In [33]:
# X_test